# Change working directory

In [ ]:
cd /usr/src/app/yolov7

In [ ]:
!pwd

# Import libraries

In [ ]:
import os, multiprocessing

# Training of model


- Change of learing rate: 
    - Edit `data/hyp.scratch.p6.yaml`
    - Change lr0 to 0.001
- Go to `train_aux.py` (line no 527). Change default value of --hyp to 'data/hyp.scratch.p6.yaml'

In [ ]:
weight = 'yolov7-w6_training.pt'
data = '../../mydataset/cfg_folders/cfg_combined_bal/combined_bal.yaml'

imgs = 1280
bs = 8
epoch = 18

name = f'train_combined_bal_{imgs}_w6_lr_001'
n = multiprocessing.cpu_count()
s = f'python train_aux.py --weights {weight} --data {data} --epoch {epoch} --img-size {imgs} {imgs} --batch-size {bs} --device 0 --workers {n} --image-weights --name {name}'
print(s)
os.system(s)

# Inference

- The following code runs two inferences. One achieves the best reported score on test-1 and another on test-2.
- Inference files (.txt) in the format asked by organiser will be saved in `runs/submissions` folder.


In [ ]:
c_1280_best = 'runs/train/train_combined_40k_14k_1280_w6_lr_001/weights/best.pt'
c_1280_e10 = 'runs/train/train_combined_40k_14k_1280_w6_lr_001/weights/epoch_010.pt'

cb_1280_best = 'runs/train/train_combined_bal_1280_w6_lr_001/weights/best.pt'
cb_1280_e10 = 'runs/train/train_combined_bal_1280_w6_lr_001/weights/epoch_010.pt'

cb_1280_aug_best = 'runs/train/train_combined_bal_14k_aug_1280_w6_lr_001/weights/best.pt'
cb_1280_aug_e10 = 'runs/train/train_combined_bal_14k_aug_1280_w6_lr_001/weights/epoch_010.pt'

weight_list = [f'{c_1280_best} {cb_1280_best} {cb_1280_e10}', f'{c_1280_best} {cb_1280_aug_best} {cb_1280_aug_e10}']
name_list = ['e_1280_c_best_cb_best_e10', 'e_1280_c_best_cb_aug_best_e10']
e = [True] * 2
test_sources = ['test_1', 'test_2']
img_list = [1280] * 2
conf = 0.01
iou_list = [0.45]
nosave_flag = True

for iou in iou_list:
    for wt, name, imgs, ensemble in zip(weight_list, name_list, img_list, e):
        if name == 'e_1280_c_best_cb_best_e10':
            test_sources = ['test_1']
        else:
            test_sources = ['test_2']

        for test_src in test_sources:
            print('----------')
            name_detect = f'{name}_c{str(int(conf*100))}_i{str(int(iou*100))}_{test_src}'
            source = f'../../mydataset/{test_src}/images'
            no_trace = '--no-trace' if ensemble else ''
            nosave = '--nosave' if nosave_flag else ''

            print(name_detect)
            
            # performing inference
            s_detect = f'python detect.py --weights {wt} --source {source} --img-size {imgs} --conf-thres {conf} --iou-thres {iou}  --device 0  {nosave} --augment --save-txt --save-conf --name {name_detect} {no_trace}'
            print(s_detect)
            os.system(s_detect)

            # for saving the results to submission format
            labelpath = f'runs/detect/{name_detect}/labels'
            s_sub = f'python runs/convert_yolo_to_subformat.py --imagepath {test_src} --labelpath {labelpath}'
            print(s_sub)
            os.system(s_sub)